<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-4-Sprint-1-Tree-Ensembles/blob/master/forColab_loans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install category_encoders

    100% |████████████████████████████████| 61kB 2.0MB/s 


In [ ]:

%matplotlib inline
from ipywidgets import interact
import pandas as pd
import numpy as np
import logging
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import category_encoders as ce

In [9]:
RAWPREFIX = "https://raw.githubusercontent.com/quinn-dougherty/DS-Unit-4-Sprint-1-Tree-Ensembles/master/loans/"

#df = pd.read_csv(RAWPREFIX+'train_features.csv')
print(df.shape)
#df.describe()


#greater_than_10k = [k for k,v in df.mean().iteritems() if v > 10**4]

todrop = ['id', # id is random
              'member_id', # all null
              'url', # all null
              'desc', # all null
              'title', # duplicate of purpose
              'grade', # duplicate of sub_grade
              'emp_title', # getting re-engineered, cardinality too high
              'zip_code' # cardinality too high
               # will be engineered into units of k
             ] + \
             greater_than_10k

many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

[i for i in todrop if i not in many_nulls]

[i for i in greater_than_10k if i not in many_nulls]

(37745, 103)


['id',
 'loan_amnt',
 'funded_amnt',
 'annual_inc',
 'revol_bal',
 'tot_cur_bal',
 'total_bal_il',
 'total_rev_hi_lim',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit']

In [20]:

def clean(dat: pd.DataFrame) -> pd.DataFrame: 
    ''' refactored Ryan H's wrangle function from lecture today into method chaining'''
    
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']
    
    greater_than_10k = [i for i in [k for k,v in dat.mean().iteritems() if v > 10**4] if i not in many_nulls]
    
    todrop = [i for i in ['id', # id is random
                          'member_id', # all null
                          'url', # all null
                          'desc', # all null
                          'title', # duplicate of purpose
                          'grade', # duplicate of sub_grade
                          'emp_title', # getting re-engineered, cardinality too high
                          'zip_code' # cardinality too high
                           # will be engineered into units of k
                          ] + \
                           greater_than_10k 
              if i not in many_nulls]

    
    
    def wrangle_sub_grade(x):
        '''Transform sub_grade from "A1" - "G5" to 1.1 - 7.5'''
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    assigns1 = {# sub_grade to ordinal
        **{'sub_grade': dat.sub_grade.apply(wrangle_sub_grade)}, # sub_grade to ordinal
        # Convert percentages from strings to floats
        **{name: dat[name].str.strip('%').astype(float) 
                 for name in ['int_rate', 'revol_util']}, # Convert percentages from strings to floats
        # Transform earliest_cr_line to an integer: how many days it's been open
        **{'earliest_cr_line': (pd.Timestamp.today() - \
                                  pd.to_datetime(dat.earliest_cr_line, infer_datetime_format=True)
                               ).dt.days},  
        # 
        **{'emp_title': dat.emp_title.str.lower()},
        # Transform features with many nulls to binary flags
        **{name: dat[name].isnull() for name in many_nulls},
        # values over 10k are too large. let's make them in units of k. 
        **{name+'_in_kUSD': dat[name] * 10**-3 for name in greater_than_10k}
              }
    
    assigns2 = {# Create features for three employee titles: teacher, manager, owner
        **{'emp_title_'+name: dat.emp_title.str.contains(name, na=False) 
                              for name in ['teacher', 'manager', 'owner']},
        # For features with few nulls, do mean imputation
        **{name: dat[name].fillna(dat[name].mean()) for name in dat.select_dtypes(include=['int', 'float', 'float64']).columns},
        
    }
    
    return (dat.assign(**assigns1)
               .assign(**assigns2)
               
               #.assign(revol_util = dat.revol_util.fillna(dat.revol_util.mean()))
               .drop(todrop, axis=1))
    
#df = clean(pd.read_csv(RAWPREFIX+'train_features.csv'))
# inscrutably, it is absolutely necessary to do this here. 
df.revol_util = df.revol_util.fillna(df.revol_util.mean()) 

#[k for k,v in df.isna().sum().iteritems() if v > 0]
df.head()


#df[[k for k,v in df.isna().sum().iteritems() if v > 0]]

,term,int_rate,installment,sub_grade,emp_length,home_ownership,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,id_in_kUSD,loan_amnt_in_kUSD,funded_amnt_in_kUSD,annual_inc_in_kUSD,revol_bal_in_kUSD,tot_cur_bal_in_kUSD,total_bal_il_in_kUSD,total_rev_hi_lim_in_kUSD,avg_cur_bal_in_kUSD,tot_hi_cred_lim_in_kUSD,total_bal_ex_mort_in_kUSD,total_bc_limit_in_kUSD,total_il_high_credit_limit_in_kUSD,emp_title_teacher,emp_title_manager,emp_title_owner
0,36 months,16.02,422.01,3.5,False,MORTGAGE,debt_consolidation,NC,12.76,0,4857,1,48.000000,82.125202,7,0,23.5,15,w,0,48.000000,Individual,125679.509734,18.895046,0,0,1,2,0,4,13.0,82.0,2,5,2962,63.0,1,1,2,9,588.0,89.3,0,0,145.0,23,1,1,1,8.0,40.675613,1.0,37.981559,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,43.373,12.0,12.0,81.0,4.912,174.783,33.668,20.9,24.969,206.618,38.580,5.5,40.863,False,False,False
1,36 months,14.07,215.54,3.3,False,RENT,debt_consolidation,CA,21.42,0,2575,1,36.951613,82.125202,10,0,59.4,12,w,0,46.062273,Individual,125679.509734,18.895046,0,0,0,1,0,2,14.0,83.0,3,8,5153,72.0,1,0,1,10,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,40.675613,6.0,37.981559,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,24.414,6.3,6.3,39.0,11.876,31.046,19.170,20.0,3.105,43.140,31.046,16.9,23.140,False,False,False
2,36 months,7.21,139.38,1.3,False,RENT,debt_consolidation,CA,2.17,0,6897,2,36.951613,82.125202,13,0,5.2,19,w,0,46.062273,Individual,125679.509734,18.895046,0,0,0,8,1,2,10.0,106.0,1,1,1196,54.0,4,0,3,3,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,40.675613,1.0,37.981559,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,46.723,4.5,4.5,78.0,1.715,35.329,33.614,33.3,2.718,65.092,35.329,23.3,31.792,False,False,False
3,36 months,9.44,384.06,2.1,False,MORTGAGE,debt_consolidation,OH,13.76,0,5891,0,36.951613,82.125202,22,0,16.3,33,w,0,46.062273,Individual,125679.509734,18.895046,0,0,4,1,0,2,13.0,67.0,6,9,6898,32.0,3,1,2,12,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,40.675613,5.0,37.981559,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,24.878,12.0,12.0,62.0,9.404,75.160,17.123,57.6,3.579,133.065,26.527,40.5,25.465,False,False,False
4,36 months,22.35,460.47,4.5,False,MORTGAGE,debt_consolidation,TX,14.04,4,6348,2,13.000000,82.125202,18,0,24.2,45,w,0,13.000000,Individual,125679.509734,18.895046,0,7218,9,2,2,4,6.0,68.0,9,10,5373,35.0,3,6,3,14,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.000000,3.0,62.000000,6,

In [67]:
[k for k,v in df.mean().iteritems() if v > 10**4]

#type(df.mean())

#df.mean().iteritems()

['loan_amnt',
 'funded_amnt',
 'revol_bal',
 'annual_inc_joint',
 'tot_cur_bal',
 'total_bal_il',
 'total_rev_hi_lim',
 'avg_cur_bal',
 'bc_open_to_buy',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'revol_bal_joint']

In [56]:
cats = list(df.select_dtypes(include=['object']).columns)
nums = list(df.select_dtypes(exclude=['object']).columns)

column_trans = ColumnTransformer(
     [('categorical', ce.BinaryEncoder(), cats),
      ('numeric', StandardScaler(), nums)],
     remainder='passthrough')


pipe = Pipeline([#('clean', FunctionTransformer(func=clean)),
                 #('scale', StandardScaler()),
                 #('encode', ce.BinaryEncoder()), 
                 ('coltrans', column_trans),
                 #('pca', PCA(n_components='mle')),
                 ('classify', LogisticRegression())])

#DecisionTreeClassifier(max_depth=5, class_weight='balanced')


X_train = clean(pd.read_csv(RAWPREFIX+'train_features.csv'))
X_test = clean(pd.read_csv(RAWPREFIX+'test_features.csv'))
y_train = pd.read_csv(RAWPREFIX+'train_labels.csv')['charged_off']
#y_test = pd.read_csv('test_labels.csv')['charged_off']
sample_submission = pd.read_csv(RAWPREFIX+'sample_submission.csv')


#cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc')

#clean(y_train).head()

pipe.fit(X_train, y_train)


#X_train.select_dtypes(include=['object']).shape

#pipe.named_steps.encode.fit_transform(X_train)

#pipe.named_steps.encode

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: ignored

In [ ]:
pipe = Pipeline([#('clean', FunctionTransformer(func=clean)),
                 ('encode', ce.BinaryEncoder()), 
                 ('classify', DecisionTreeClassifier(max_depth=5, class_weight='balanced'))])


In [47]:
X_train.annual_income_in_thousands_USD

0        81.000  
1        39.000  
2        78.000  
3        62.000  
4        70.000  
5        100.000 
6        130.000 
7        87.000  
8        73.752  
9        80.000  
10       190.000 
11       1250.000
12       93.000  
13       34.000  
14       327.843 
15       142.000 
16       50.500  
17       100.000 
18       33.800  
19       46.874  
20       65.000  
21       67.000  
22       75.000  
23       36.200  
24       76.040  
25       15.000  
26       135.000 
27       85.000  
28       20.000  
29       112.200 
30       145.000 
31       100.000 
32       65.000  
33       56.000  
34       150.000 
35       185.000 
36       55.000  
37       65.000  
38       92.500  
39       88.000  
40       69.000  
41       69.000  
42       80.000  
43       73.000  
44       55.400  
45       42.000  
46       45.000  
47       30.000  
48       96.000  
49       65.000  
50       66.000  
51       40.000  
52       25.000  
53       290.000 
54       46.000  
55       5

In [33]:
X_train.

,loan_amnt,funded_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,emp_title_teacher,emp_title_manager,emp_title_owner
0,12000,12000,36 months,16.02,422.01,3.5,False,MORTGAGE,81000.0,debt_consolidation,NC,12.76,0,4857,1,48.000000,82.125202,7,0,4912,23.5,15,w,0,48.000000,Individual,125679.509734,18.895046,0,0,174783,1,2,0,4,13.000000,33668,82.000000,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.000000,89.300000,0,0,145.000000,23,1,1,1,8.000000,40.675613,1.000000,37.981559,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.000000,0,0,206618,38580,5500,40863,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,False,False,False
1,6300,6300,36 months,14.07,215.54,3.3,False,RENT,39000.0,debt_consolidation,CA,21.42,0,2575,1,36.951613,82.125202,10,0,11876,59.4,12,w,0,46.062273,Individual,125679.509734,18.895046,0,0,31046,0,1,0,2,14.000000,19170,83.000000,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.000000,59.300000,0,0,17.000000,72,8,8,0,8.000000,40.675613,6.000000,37.981559,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.900000,0,0,43140,31046,16900,23140,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,False,False,False
2,4500,4500,36 months,7.21,139.38,1.3,False,RENT,78000.0,debt_consolidation,CA,2.17,0,6897,2,36.951613,82.125202,13,0,1715,5.2,19,w,0,46.062273,Individual,125679.509734,18.895046,0,0,35329,0,8,1,2,10.000000,33614,106.000000,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.000000,7.400000,0,0,166.000000,218,10,10,0,31.000000,40.675613,1.000000,37.981559,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.000000,0,0,65092,35329,23300,31792,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,False,False,False
3,12000,12000,36 months,9.44,384.06,2.1,False,MORTGAGE,62000.0,debt_consolidation,OH,13.76,0,5891,0,36.951613,82.125202,22,0,9404,16.3,33,w,0,46.062273,Individual,125679.509734,18.895046,0,0,75160,4,1,0,2,13.000000,17123,67.000000,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.000000,17.000000,0,0,162.000000,179,1,1,1,1.000000,40.675613,5.000000,37.981559,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.000000,0,0,133065,26527,40500,25465,33654.305192,True,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.07366,36.332134,Cash,False,False,False
4,12000,12000,36 months,22.35,460.47,4.5,False,MORTGAGE,70000.0,debt_consolidation,TX,14.04,4,6348,2,13.000000,82.125202,18,0,17895,24.2,45,w,0,13.000000,Individual,125679.509734,18.895046,0,7218,113702,9,2,2,4,6.000000,16711,68.000000,9,10,